## Deliverable 2. Create a Customer Travel Destinations Map.

In [94]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [95]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/Weatherpy_Database.csv')
city_data_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,40.66,81,40,11.50,scattered clouds
1,1,Rikitea,PF,-23.1203,-134.9692,70.81,62,76,21.12,broken clouds
2,2,Codrington,AU,-38.2667,141.9667,41.83,98,60,6.85,broken clouds
3,3,Pritzwalk,DE,53.1494,12.1741,64.58,77,100,7.54,overcast clouds
4,4,Jamestown,US,42.0970,-79.2353,80.19,68,0,9.22,clear sky
5,5,Kapaa,US,22.0752,-159.3190,81.12,77,20,5.75,few clouds
6,6,Sola,VU,-13.8833,167.5500,80.65,76,62,18.52,broken clouds
7,7,Bluff,NZ,-46.6000,168.3333,48.45,91,77,6.46,broken clouds
8,8,Broken Hill,AU,-31.9500,141.4333,43.30,84,0,12.68,clear sky
9,9,Thompson,CA,55.7435,-97.8558,57.36,100,100,13.80,mist


In [127]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [128]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,70.81,62,76,21.12,broken clouds
3,3,Pritzwalk,DE,53.1494,12.1741,64.58,77,100,7.54,overcast clouds
10,10,Luderitz,NaN,-26.6481,15.1594,61.99,63,86,5.08,overcast clouds
12,12,Kysyl-Syr,RU,63.8986,122.7617,60.66,98,100,9.08,light rain
16,16,Wexford,IE,52.3342,-6.4575,63.90,56,100,1.99,overcast clouds
18,18,Tura,IN,25.5198,90.2201,74.70,96,100,3.36,light rain
20,20,Puerto Colombia,CO,10.9878,-74.9547,77.09,77,100,13.40,overcast clouds
22,22,Dingle,PH,10.9995,122.6711,74.98,91,100,3.51,overcast clouds
26,26,Lardos,GR,36.0942,28.0157,78.94,82,0,13.71,clear sky
27,27,Smila,UA,49.2224,31.8871,68.90,85,30,8.61,scattered clouds


In [129]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                336
City                   336
Country                334
Lat                    336
Lng                    336
Max Temp               336
Humidity               336
Cloudiness             336
Wind Speed             336
Current Description    336
dtype: int64

In [130]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")
clean_df.count()

City ID                334
City                   334
Country                334
Lat                    334
Lng                    334
Max Temp               334
Humidity               334
Cloudiness             334
Wind Speed             334
Current Description    334
dtype: int64

In [131]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.81,broken clouds,-23.1203,-134.9692,
3,Pritzwalk,DE,64.58,overcast clouds,53.1494,12.1741,
12,Kysyl-Syr,RU,60.66,light rain,63.8986,122.7617,
16,Wexford,IE,63.90,overcast clouds,52.3342,-6.4575,
18,Tura,IN,74.70,light rain,25.5198,90.2201,
20,Puerto Colombia,CO,77.09,overcast clouds,10.9878,-74.9547,
22,Dingle,PH,74.98,overcast clouds,10.9995,122.6711,
26,Lardos,GR,78.94,clear sky,36.0942,28.0157,
27,Smila,UA,68.90,scattered clouds,49.2224,31.8871,
29,Tuktoyaktuk,CA,71.60,few clouds,69.4541,-133.0374,


In [132]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.....skipping.")

Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not 

In [133]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.81,broken clouds,-23.1203,-134.9692,People ThankYou
3,Pritzwalk,DE,64.58,overcast clouds,53.1494,12.1741,Landhotel & Brauhaus Prignitzer Hof
12,Kysyl-Syr,RU,60.66,light rain,63.8986,122.7617,NaN
16,Wexford,IE,63.90,overcast clouds,52.3342,-6.4575,Talbot Hotel Wexford
18,Tura,IN,74.70,light rain,25.5198,90.2201,Hotel RIKMAN Continental
20,Puerto Colombia,CO,77.09,overcast clouds,10.9878,-74.9547,Hotel El Emigrante
22,Dingle,PH,74.98,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
26,Lardos,GR,78.94,clear sky,36.0942,28.0157,Lindos Princess Beach Hotel
27,Smila,UA,68.90,scattered clouds,49.2224,31.8871,Kolo Zhyttya
29,Tuktoyaktuk,CA,71.60,few clouds,69.4541,-133.0374,End of The Road Ltd.


In [134]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.81,broken clouds,-23.1203,-134.9692,People ThankYou
3,Pritzwalk,DE,64.58,overcast clouds,53.1494,12.1741,Landhotel & Brauhaus Prignitzer Hof
16,Wexford,IE,63.90,overcast clouds,52.3342,-6.4575,Talbot Hotel Wexford
18,Tura,IN,74.70,light rain,25.5198,90.2201,Hotel RIKMAN Continental
20,Puerto Colombia,CO,77.09,overcast clouds,10.9878,-74.9547,Hotel El Emigrante
22,Dingle,PH,74.98,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
26,Lardos,GR,78.94,clear sky,36.0942,28.0157,Lindos Princess Beach Hotel
27,Smila,UA,68.90,scattered clouds,49.2224,31.8871,Kolo Zhyttya
29,Tuktoyaktuk,CA,71.60,few clouds,69.4541,-133.0374,End of The Road Ltd.
33,Bengkulu,ID,73.53,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu


In [135]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [138]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name </dt><dd>{Hotel Name}</dd>
<dt>City </dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [141]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))